In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import math
from shapely.geometry import Point
import pickle
import os
%matplotlib inline

In [3]:
path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
#path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [4]:
uber_path = path+"Uber Data/"

## Create Traffic Measures

In [5]:
def parse_and_add(new_filename,combined_df, step):
    df = pd.read_csv(uber_path + new_filename)
    df['Date/Time'] = pd.to_datetime(df['Date/Time'], format='%m/%d/%Y %H:%M:%S')
    
    to_bin = lambda x: np.round(x / step) * step
    df["Lat_bin"] = df.Lat.map(to_bin)
    df["Long_bin"] = df.Lon.map(to_bin)
    df['hour']=df['Date/Time'].dt.hour
    df['dayofweek']=df['Date/Time'].dt.dayofweek
    grouped_df = df.groupby(['Lat_bin','Long_bin','hour','dayofweek'])['Base'].count()
    del df
    
    if type(combined_df) == type(None):
        return grouped_df
    else:
        return combined_df.combine(grouped_df,lambda x,y: x+y, fill_value=0)

In [6]:
file_list = ['uber-raw-data-apr14.csv',
 'uber-raw-data-aug14.csv',
 'uber-raw-data-jul14.csv',
 'uber-raw-data-jun14.csv',
 'uber-raw-data-may14.csv',
 'uber-raw-data-sep14.csv']

In [12]:
step=0.005
combined_df = None
for filename in file_list:
    print(filename)
    combined_df = parse_and_add(filename,combined_df, step)
combined_df.name = 'Uber_Count'
pickle.dump(combined_df, open('./data/hourly_traffic_intensity_'+str(step)+'.pkl','wb'))

uber-raw-data-apr14.csv
uber-raw-data-aug14.csv
uber-raw-data-jul14.csv
uber-raw-data-jun14.csv
uber-raw-data-may14.csv
uber-raw-data-sep14.csv


## Join Traffic Measures

In [8]:
def join_traffic_to_collision(collisions_path, step):
    collisions = pickle.load(open(collisions_path,'rb'), encoding='latin1')
    
    to_bin = lambda x: np.round(x / step) * step
    collisions["Lat_bin"] = collisions.LATITUDE.map(to_bin)
    collisions["Long_bin"] = collisions.LONGITUDE.map(to_bin)
    collisions['hour'] = collisions.DATE_TIME.dt.hour
    collisions['dayofweek'] = collisions.DATE_TIME.dt.dayofweek
    collisions = collisions[['UNIQUE KEY','hour','dayofweek','Long_bin','Lat_bin']].dropna()
    
    df = pickle.load(open('./data/hourly_traffic_intensity_'+str(step)+'.pkl','rb'))

    joined = collisions.join(df, on = ['Lat_bin','Long_bin','hour','dayofweek']).fillna(0)
    joined.index=joined['UNIQUE KEY']
    return joined[joined.columns[-1]]

In [14]:
step = 0.003
pickle.dump(join_traffic_to_collision(path + 'data_for_joining/collisions.pkl', step), open(path+'data_for_joining/traffic_joined_'+str(step)+'.pkl','wb'))